In [1]:
import functions

functions.get_artist_info("Vincent van Gogh")

{'name': 'Vincent van Gogh',
 'birth_place': 'The Hague',
 'birth_date': '1789-02-11T00:00:00Z',
 'death_date': '1874-05-07T00:00:00Z',
 'death_place': 'Breda',
 'work_locations': [{'location': 'Saint-Rémy-de-Provence',
   'start_time': '1889-05-01T00:00:00Z',
   'end_time': '1890-05-01T00:00:00Z',
   'point_in_time': None},
  {'location': 'The Hague',
   'start_time': '1881-12-01T00:00:00Z',
   'end_time': '1883-09-01T00:00:00Z',
   'point_in_time': None},
  {'location': 'Ramsgate',
   'start_time': '1876-04-01T00:00:00Z',
   'end_time': '1876-12-01T00:00:00Z',
   'point_in_time': None},
  {'location': 'City of Brussels',
   'start_time': '1880-10-01T00:00:00Z',
   'end_time': '1881-04-01T00:00:00Z',
   'point_in_time': None},
  {'location': 'Etten-Leur',
   'start_time': '1881-04-01T00:00:00Z',
   'end_time': '1881-12-01T00:00:00Z',
   'point_in_time': None},
  {'location': 'Dordrecht',
   'start_time': '1877-01-01T00:00:00Z',
   'end_time': '1877-04-01T00:00:00Z',
   'point_in_time'

In [2]:
functions.get_artist_locations("Vincent van Gogh")

{'name': 'Vincent van Gogh',
 'birth_place': 'The Hague',
 'death_place': 'Breda',
 'work_locations': [{'location': 'Saint-Rémy-de-Provence',
   'start_time': '1889-05-01T00:00:00Z',
   'end_time': '1890-05-01T00:00:00Z',
   'point_in_time': None},
  {'location': 'The Hague',
   'start_time': '1881-12-01T00:00:00Z',
   'end_time': '1883-09-01T00:00:00Z',
   'point_in_time': None},
  {'location': 'Ramsgate',
   'start_time': '1876-04-01T00:00:00Z',
   'end_time': '1876-12-01T00:00:00Z',
   'point_in_time': None},
  {'location': 'City of Brussels',
   'start_time': '1880-10-01T00:00:00Z',
   'end_time': '1881-04-01T00:00:00Z',
   'point_in_time': None},
  {'location': 'Etten-Leur',
   'start_time': '1881-04-01T00:00:00Z',
   'end_time': '1881-12-01T00:00:00Z',
   'point_in_time': None},
  {'location': 'Dordrecht',
   'start_time': '1877-01-01T00:00:00Z',
   'end_time': '1877-04-01T00:00:00Z',
   'point_in_time': None},
  {'location': 'Nuenen',
   'start_time': '1883-12-01T00:00:00Z',
   

https://www.wikidata.org/wiki/Q5582
https://www.wikidata.org/wiki/Property:P580